<h3 align="center">Guide for detecting lines and colors in an image</h3>  

<img src= "sources/processing_detect_line_image.png" />

In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
%matplotlib qt5

In [55]:
# Open image 
path = r'D:\UDEA-MAESTRIA\Pasantia\Universidad\Script\Scaner/'
pathSave = r'D:\UDEA-MAESTRIA\Pasantia\Universidad\Script/Areas/'
imageName = '12.jpg'
image = cv2.imread(path + imageName) 
plt.imshow(image)

In [56]:
# Split images in the same page

middle = int(len(image)/2) 
image1 = image[:middle]
plt.imshow(image1)

image2 = image[middle:]
plt.figure()
plt.imshow(image2)



In [59]:
# Select image for processing, binarization and dilate for connectiing some points 

setImage = image2
threshold = 170

grayImage = cv2.cvtColor(setImage, cv2.COLOR_BGR2GRAY)
_, binaryImage = cv2.threshold(grayImage, threshold, 255, cv2.THRESH_BINARY)
kernel = np.ones((3, 3), np.uint8)
dilateImage = cv2.dilate(~binaryImage, kernel, iterations=3)
plt.imshow(dilateImage)


cv2.imwrite( pathSave + 'original.png', setImage)


True

In [60]:
# Get exteral contour and separate main circle 

r,c = np.shape(grayImage)
ext = np.zeros((r,c), np.dtype('uint8'))
    
contour,_ = cv2.findContours(dilateImage, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
ext = cv2.drawContours(ext, contour, -1, 255, -1)
plt.imshow(~ext)

    

In [61]:
# Mask with external contour 
imaIn = ~dilateImage*ext
plt.imshow(imaIn)

In [62]:
# Identify secction and put specific lables for each one

sections, labels = cv2.connectedComponents(imaIn)
print('Number of sections: ' + str(sections-1))
plt.imshow(labels)

Number of sections: 5


In [11]:
plt.imshow(setImage)

In [64]:
# Calculate pixels in each region and make a erosion and dilation to get only the line save the image
for i in range(sections):
    area = np.sum(labels==i)
    print ('Area ', str(area))
    if (area > 20):
        section =  labels.copy()
        section[section != i] = 0
        section[section == i] = 255
        new_image = section.astype(np.uint8)
        kernel = np.ones((15, 15), np.uint8)
        dilateImage = cv2.dilate(new_image, kernel, iterations=5)
        erodeImage = cv2.erode(new_image, kernel, iterations=5)
        newImage = cv2.bitwise_and(setImage, setImage, mask=dilateImage)
        newImage2 = cv2.bitwise_and(newImage, newImage, mask=~erodeImage)
        cv2.imwrite( pathSave + str(i) +'.png', newImage2)




Area  13551692
Area  2689422
Area  640533
Area  204352
Area  218896
Area  89825


In [69]:
# In each area it is necesarry to identify which is the lines's color. In this part it is possible to 
#get the histogram for the secction

pathAreas = r'D:\UDEA-MAESTRIA\Pasantia\Universidad\Script/Areas/'

imageName= '5.png'
image = cv2.imread(pathAreas + imageName) 
imageHSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

Matriz = imageHSV[:,:,0]
plt.figure()
plt.imshow(Matriz)

plt.figure()
histr = cv2.calcHist([imageHSV],[0],None,[360],[0,360])
plt.plot(histr,color = col)
plt.xlim([0,200])
plt.ylim([0,25000])
plt.show()
